<a href="https://colab.research.google.com/github/Andresf-Asprilla/Modelo-segmentacion-APCIVMAPCAs/blob/main/Escalado_Intensidad_Espaciado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ejecuta el siguiente código una sola vez haciendo clic dentro de la celda y luego presionando Control + Enter.

Antes de ejecutar el siguiente script, debe instalar las siguientes dependencias:
```
!pip install -q "monai-weekly[gdown, nibabel, tqdm, itk]"
!pip install SimpleITK
```



In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import SimpleITK as sitk
import nibabel as nib
import monai.transforms as tf
import os


def NormalizacionyEspaciado(ruta_directorio, ruta_salida, nuevo_espaciado):
    """
    Cambia el espaciado de todas las imágenes en un directorio sin modificar sus dimensiones ni el tamaño de las estructuras.
    :param ruta_directorio: Ruta al directorio que contiene los archivos NIfTI de entrada.
    :param ruta_salida: Ruta al directorio donde se guardarán los archivos procesados.
    :param nuevo_espaciado: Nuevo espaciado como lista [x, y, z].
    """
    os.makedirs(ruta_salida, exist_ok=True)
    for archivo in os.listdir(ruta_directorio):
        if archivo.endswith('.nii') or archivo.endswith('.nii.gz'):
            ruta_entrada = os.path.join(ruta_directorio, archivo)
            ruta_salida_archivo = os.path.join(ruta_salida, archivo)
            # Aplicar el cambio de espaciado
            imagen = sitk.ReadImage(ruta_entrada)
            # Obtener el tamaño, espaciado y dirección originales
            tamaño_original = imagen.GetSize()
            espaciado_original = imagen.GetSpacing()
            direccion_original = imagen.GetDirection()
            print(f"Espaciado original: {espaciado_original}")
            print(f"Tamaño original: {tamaño_original}")
            print(f"Dirección original: {direccion_original}")
            # Ajustar el espaciado sin modificar las dimensiones
            imagen.SetSpacing(nuevo_espaciado)
            # Calcular el nuevo origen para preservar la posición física
            origen_actual = imagen.GetOrigin()
            nuevo_origen = [
                o + (s1 - s2) * (t - 1) / (2 * t) for o, s1, s2, t in zip(origen_actual,
                                                                          espaciado_original,
                                                                          nuevo_espaciado,
                                                                          tamaño_original)
            ]
            imagen.SetOrigin(nuevo_origen)
            # Reescalar la imagen para mantener el tamaño físico de las estructuras
            escala = [s1 / s2 for s1, s2 in zip(espaciado_original, nuevo_espaciado)]
            nuevo_tamaño = [int(round(t * e)) for t, e in zip(tamaño_original, escala)]
            imagen_resampleada = sitk.Resample(imagen, tamaño_original, sitk.Transform(), sitk.sitkLinear,
                                              nuevo_origen, nuevo_espaciado, direccion_original,
                                              0, imagen.GetPixelID())

            # Guardar la nueva imagen
            sitk.WriteImage(imagen_resampleada, ruta_salida_archivo)
            print(f"Nuevo espaciado aplicado: {nuevo_espaciado}")
            # Normalización de intensidad
            nifti_imagen = nib.load(ruta_salida_archivo)
            datos_imagen = nifti_imagen.get_fdata()
            affine = nifti_imagen.affine
            # Escalar intensidad
            escalar_intensidad = tf.ScaleIntensity()
            imagen_normalizada = escalar_intensidad(datos_imagen)
            # Convertir a numpy y guardar la imagen normalizada sobreescribiendo la imagen final
            imagen_normalizada = imagen_normalizada.numpy()
            # Guardar la imagen procesada (reespaciada y normalizada) en el archivo final
            nib.save(nib.Nifti1Image(imagen_normalizada, affine), ruta_salida_archivo)
            print("Imagen reespaciada y normalizada guardada")



In [ ]:
# Uso para un directorio
ruta_directorio = "/content/drive/MyDrive/Colab Notebooks/Imagenes_redimencionadas_Respaciadas/ImagesEscalarRedimecionar"# cambia esta direccion por la carpeta donde se encuentra ubicadas las imagenes dicom en tu drive.
ruta_salida = "/content/drive/MyDrive/Colab Notebooks/Imagenes_redimencionadas_Respaciadas/EscaladasRedimecionadas" # Cambia esta dirección por la carpeta  donde deseas que las imágenes reescaladas y espacadas se almacenen.
nuevo_espaciado = [1.0, 1.0, 1.0]  # Espaciado deseado en mm

NormalizacionyEspaciado(ruta_directorio, ruta_salida, nuevo_espaciado)
